In [1]:
from gol_apis import RosterActivitiesService
from gol_apis import RosterTagService
from gol_apis import CheckScheduleService
from gol_apis import CrewMemberInfoService
import api_tools
from api_tools.setup import setup_crew_info_dataframe
from api_tools.setup import setup_checks_dataframe
from api_tools.setup import setup_tag_dataframe
from api_tools.setup import setup_activities_dataframe

from gol_logger import Logger
import pandas as pd
import multiprocessing as mp
import requests, time
from datetime import datetime as dt
from datetime import timedelta as td
from utils import funcs
from utils.progress import step
from utils.progress import BAR
from itertools import chain

from modules.generate_data import get_data_neolude
from modules.generate_data import get_rosters
from modules.generate_data import get_crew_info
from modules.generate_data import get_checks

from modules.mail import send_checks_mail

import sys
from config.variables import user

In [2]:
training_code = 'CFI Inicial'

In [3]:
# Dados Crew Info
crew = pd.read_csv('data/crew/crew_info.csv', sep=';', encoding='utf-8-sig')
crew.cif = crew.cif.astype(int).astype(str).str.zfill(8)
crew = crew[crew['funcao'].isin(['CMT', 'COP'])]
crew = crew[['cif', 'guerra', 'funcao', 'cargo', 'base']]
crew

,cif,guerra,funcao,cargo,base
0,00000020,BORIO,CMT,IBX,CGH
1,00000021,GRABLER,CMT,IBX,GRU
2,00000022,DANTAS,CMT,IBX,CGH
3,00000024,BORBA,CMT,NaN,GRU
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH
...,...,...,...,...,...
5473,99999913,PIAZZA,CMT,IB,GRU
5474,99999916,GALENO CABRAL,CMT,IB,GRU
5475,99999917,EDWARD,CMT,IB,GRU
5476,99999918,CESIDIO,CMT,IB,GRU


In [4]:
# Header para filtrar dados do Neolude
header_neolude = ['RegistrationNumber',
                  'UserName',
                  'BusinessUnitName',
                  'Post',
                  'CPF',
                  'Email',
                  'CourseName',
                  'CourseUserStatus',
                  'GradeFormatted',
                  'LastAccessDate',
                  'CreatedDate',
                  'CompletionDate']

In [6]:
# Dados Neolude
training = pd.read_csv(f'data/neolude/{training_code}.csv', sep=';', encoding='utf-8-sig')
training = training[header_neolude]
training['RegistrationNumber'] = training['RegistrationNumber'].fillna(0)
training['RegistrationNumber'] = training['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training['LastAccessDate'] = pd.to_datetime(training['LastAccessDate'])
training['CreatedDate'] = pd.to_datetime(training['CreatedDate'])
training['CompletionDate'] = pd.to_datetime(training['CompletionDate'])
training.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name','status', 'nota', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao']
training = training[['cif', 'course_name','status', 'nota', 'data_conclusao']]
training = training.sort_values(by=['cif','data_conclusao'])
training = training.drop_duplicates(subset = ['cif'], keep = 'last')
training

,cif,course_name,status,nota,data_conclusao
93,00000000,Tripulação Técnica - CFI_Inicial (Curso de For...,Concluído,"0,00",2017-01-06
257,00000020,Tripulação Técnica - CFI_Inicial (Curso de For...,Concluído,"0,00",2010-09-16
249,00000021,Tripulação Técnica - CFI_Inicial (Curso de For...,Concluído,"0,00",2010-08-26
188,00000026,Tripulação Técnica - CFI_Inicial (Curso de For...,Concluído,"0,00",2010-08-26
156,00000029,Tripulação Técnica - CFI_Inicial (Curso de For...,Concluído,"0,00",2010-09-16
...,...,...,...,...,...
251,00035306,Tripulação Técnica - CFI_Inicial (Curso de For...,Concluído,"0,00",2020-01-07
76,00035317,Tripulação Técnica - CFI_Inicial (Curso de For...,Concluído,"0,00",2020-01-07
231,00037175,Tripulação Técnica - CFI_Inicial (Curso de For...,Concluído,"0,00",2015-11-17
237,00041645,Tripulação Técnica - CFI_Inicial (Curso de For...,Concluído,"0,00",2020-01-07


In [ ]:
# Dados de Rosters
rosters = funcs.generate_data('data/rosters/')
rosters.cif = rosters.cif.astype(str).str.zfill(8)

In [ ]:
inss = rosters.copy()
inss['dep'] = pd.to_datetime(inss['dep'], format='%Y-%m-%d')
inss['dep'] = inss['dep'].dt.floor('D')
inss = inss[(inss['code'] == 'INSS') & (inss['dep'] == pd.to_datetime(dt.date(dt.now())))]
inss = inss[['cif', 'code']]
inss.columns = ['cif', 'inss']
inss

In [ ]:
lnr = rosters.copy()
lnr['dep'] = pd.to_datetime(lnr['dep'], format='%Y-%m-%d')
lnr['dep'] = lnr['dep'].dt.floor('D')
lnr = lnr[(lnr['code'] == 'LIC-SREM') & (lnr['dep'] == pd.to_datetime(dt.date(dt.now())))]
lnr = lnr[['cif', 'code']]
lnr.columns = ['cif', 'lnr']
lnr

In [ ]:
rosters = rosters[rosters['code'].isin(funcs.training_filter(training_code))]
rosters = rosters.sort_values(by=['cif','dep'])
rosters = rosters[['cif', 'code', 'fr', 'dep']]
rosters = rosters.drop_duplicates(subset = ['cif'], keep = 'last')
rosters

In [ ]:
# Dados de Checks
checks = pd.read_csv('data/checks/check_schedule.csv', sep=';', encoding='utf-8-sig')
checks.cif = checks.cif.astype(str).str.zfill(8)
checks = checks[checks['checkTypeCode'] == training_code]
checks = checks[['cif', 'checkTypeCode', 'expiryDt']]
checks

In [ ]:
# Merge dos dados
data_all = pd.merge(crew, checks[['cif', 'checkTypeCode', 'expiryDt']], on='cif', how='left')
data_all = data_all.merge(training, on='cif', how='left')
data_all = data_all.merge(rosters[['cif', 'code', 'dep']], on='cif', how='left')
data_all = data_all.merge(inss, on='cif', how='left')
data_all = data_all.merge(lnr, on='cif', how='left')
data_all

In [ ]:
# Formatação do dataFrame de Dados
validation = data_all.copy()

# Ajuste de data do vencimento
validation['expiryDt'] = pd.to_datetime(validation['expiryDt'], format='%Y-%m-%d')
validation['expiryDt'] = validation['expiryDt'].dt.floor('D')

validation['dep'] = validation['dep'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['dep'] = pd.to_datetime(validation['dep'], format='%Y-%m-%d')
validation['dep'] = validation['dep'].dt.floor('D')

validation['data_conclusao'] = validation['data_conclusao'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao'] = pd.to_datetime(validation['data_conclusao'], format='%Y-%m-%d')
validation['data_conclusao'] = validation['data_conclusao'].dt.floor('D')

#validation = validation[validation['expiryDt'] < (pd.to_datetime(dt.date(dt.now())) + pd.offsets.MonthBegin(0))]
validation['tempo_de_conclusao (dias)'] = pd.to_datetime(dt.date(dt.now())) - validation['data_conclusao']

validation['tempo_de_conclusao (dias)'] = validation['tempo_de_conclusao (dias)'].dt.days

validation = validation[validation['dep'] < (pd.to_datetime(dt.date(dt.now())))]
validation['atualizar'] = (validation['status'] == 'Concluído') & (validation['tempo_de_conclusao (dias)'] < 90)
validation

In [ ]:
validation.to_csv(f'./data/output/{training_code}.csv', index=False, sep=';', encoding='utf-8-sig')

In [ ]:
# Arquivo de import
# fill table
df_atualizar = validation[validation['atualizar']]

df_atualizar

In [ ]:
df_import = df[['cif', 'checkTypeCode', 'data_conclusao']]

# fill table

data = df_import.copy()

data['update'] = 'U'
data['id'] = df_['cif'].astype(str).str.zfill(8)
data['type'] = 'ARP'
data['issue'] = df_['data_conclusao'].dt.strftime('%Y%m%d') + '0000'
data['expiry'] = (df_['data_conclusao'] + td(days=365)).dt.strftime('%Y%m%d') + '0000'
data['extended'] = (df_['data_conclusao'] + td(days=365)).dt.strftime('%Y%m%d') + '0000'
data['failed'] = 'N'
data['do_not_plan_before'] = (df_['data_conclusao'] + td(days=365)).dt.strftime('%Y%m%d') + '0000'
data['zero'] = '0'
data['remark'] = ''
data['check_comment'] = ''
data['confirmed'] = 'Y'
data['paper_recieved'] = 'Y'

# remove fields from original file
cols = [
    'update',
    'cif',
    'type',
    'issue',
    'expiry',
    'extended',
    'failed',
    'do_not_plan_before',
    'zero',
    'remark',
    'check_comment',
    'confirmed',
    'paper_recieved'  
]

data = data[cols].copy()

data.to_csv('./data/output/crm_import_arp.dat', sep='|', header=None, index=False, line_terminator='\r\n')

data

In [ ]:
out_email = validation.copy()

out_email = out_email[~out_email['inss'].isin(['INSS'])]
out_email = out_email[~out_email['lnr'].isin(['LIC-SREM'])]

out_email = out_email[['cif',
                       'guerra',
                       'funcao',
                       'base',
                       'checkTypeCode',
                       'expiryDt', 
                       'status', 
                       'nota', 
                       'data_conclusao',
                       'code',
                       'dep',
                       'tempo_de_conclusao (dias)',
                       'atualizar']]

out_email['expiryDt'] = out_email['expiryDt'].dt.strftime('%d/%m/%Y')
out_email['data_conclusao'] = out_email['data_conclusao'].dt.strftime('%d/%m/%Y')
out_email['dep'] = out_email['dep'].dt.strftime('%d/%m/%Y')
out_email['status'] = out_email['status'].fillna('')
out_email['nota'] = out_email['nota'].fillna('')

In [ ]:
send_checks_mail(out_email, user['mail'], user['password'], user['emails'])